# Workspace - Challenge 1

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from lumen.io import read_events
from lumen.io import read_positions
from lumen.util import extract_device_events

from lumen.device import Device

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e19bc3ac-22fd-4ca3-9f23-d099570b4b40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>